# Tratamento de dados de amaciamento por Rolling Window
Versão atual: 0.1

Este é um script exemplificando o tratamento de dados do amaciamento de compressores por rolling window para aplicações de Machine Learning.

Criado em: 12/02/2021, Gabriel Thaler
Última modificação: 12/02/2021, Gabriel Thaler

In [24]:
import pandas as pd
import numpy as np
import math

# Funções

In [25]:
def seleciona_dados(ensaio_raw: pd.DataFrame, grandezas: list[str], tMin: int = 0, tMax: int = math.inf) -> pd.DataFrame:
    """Seleciona os dados desejados de uma unidade.

    Parâmetros
    ----------
    ensaio_raw : pd.DataFrame
        Dados brutos de um ensaio

    grandezas: list[str]
        Lista com o nome das grandezas desejadas. Nomes disponíveis:
                'CorrenteRMS'               'CorrenteVariancia'            'CorrenteCurtose'
            'VibracaoCalotaInferiorRMS' 'VibracaoCalotaInferiorCurtose' 'VibracaoCalotaInferiorVariancia'
            'VibracaoCalotaSuperiorRMS' 'VibracaoCalotaSuperiorCurtose' 'VibracaoCalotaSuperiorVariancia'
                    'Vazao'

    tMin: int
        Primeiro instante desejado [h]

    tMax: int
        Último instante desejado [h]

    Retorna
    -------
    dados_selecionados: pd.DataFrame
        Dados selecionados do ensaio
    """

    dados_selecionados = pd.DataFrame() # Inicializa a variável para os dados selecionados
    dados_selecionados["Tempo"] = ensaio_raw["Tempo"] # Copia a coluna do tempo
    dados_selecionados["Amaciado"] = ensaio_raw["Amaciado"] # Copia a coluna do amaciamento

    for gr in grandezas: # Para cada grandeza na lista
        dados_selecionados[gr] = ensaio_raw[gr] # Copia a grandeza para o novo DataFrame

    # Corta os dados de acordo com as guias (default: 0 e Inf)
    dados_selecionados = dados_selecionados[(dados_selecionados.Tempo<=tMax) & (dados_selecionados.Tempo>=tMin)]

    return dados_selecionados

In [26]:
def rolling_window(ensaio: pd.DataFrame, N_instantes: int = 1, D_espacamento: int = 1, M_janela: int = 1) -> pd.DataFrame:
    """Aplica filtro de médias móveis e rolling window nos dados de uma unidade.

    Parâmetros
    ----------
    ensaio : pd.DataFrame
        Dados de um ensaio, já selecionados pela função seleciona_dados

    N_instantes: int
        Valor N, número de elementos do vetor de predição utilizados na formatação

    D_espacamento: int
        Valor D, espaçamento temporal entre features

    M_janela: int
        Valor M, largura da janela do filtro de médias móveis

    Retorna
    -------
    dados_tratados: pd.DataFrame
        Dados tratados do ensaio
    """

    dados_tratados = pd.DataFrame() # Inicializa a variável para os dados tratados
    dados_tratados["Tempo"] = ensaio["Tempo"] # Extrai o tempo do ensaio para a nova variável
    dados_tratados["Amaciado"] = ensaio["Amaciado"] # Extrai o amaciamento do ensaio para a nova variável
    ensaio = ensaio.drop(["Tempo","Amaciado"], axis = 1) # Exclui a coluna do tempo e do amaciamento
    
    for gr in ensaio.columns: # Para cada grandeza no DataFrame

        data_filtered = ensaio[gr].rolling(M_janela, min_periods=1).mean() # Aplica o filtro de médias móveis nos dados

        for n in range(N_instantes): # Para cada uma das repetições espaçadas
            dados_tratados[f'{gr}(K-{D_espacamento*n})'] = data_filtered.shift(D_espacamento*n) # Aplica o shift nos dados filtrados, e salva em uma nova coluna
            # PS: esse [f'{gr}... é uma f-string, usada pra fazer operações com strings

    dados_tratados = dados_tratados.dropna() # Exclui as linhas com NaN, deixando assim apenas as linhas válidas (k>(N-1)*D)

    return dados_tratados

In [38]:
def trata_conjunto(ensaios: list[dict], grandezas: list[str], N_instantes: int = 1, D_espacamento: int = 1, M_janela: int = 1, tMin: int = 0, tMax: int = math.inf) -> pd.DataFrame:
    """Trata todo um conjunto de unidades por rolling window, retornando o dataset final

    Parâmetros
    ----------
    ensaios : list[dict]
        Lista de dicionários, com cada entrada contendo:
            abrev: Nome da unidade (Modelo e número). Importante para separar unidades com problema em algumas grandezas.
            ensaios: Lista de DataFrames, contendo todos os dados brutos do arquivo .csv dos ensaios

    grandezas: list[str]
        Lista com o nome das grandezas desejadas. Nomes disponíveis:
                'CorrenteRMS'               'CorrenteVariancia'            'CorrenteCurtose'
            'VibracaoCalotaInferiorRMS' 'VibracaoCalotaInferiorCurtose' 'VibracaoCalotaInferiorVariancia'
            'VibracaoCalotaSuperiorRMS' 'VibracaoCalotaSuperiorCurtose' 'VibracaoCalotaSuperiorVariancia'
                    'Vazao'

    N_instantes: int
        Valor N, número de elementos do vetor de predição utilizados na formatação

    D_espacamento: int
        Valor D, espaçamento temporal entre features

    M_janela: int
        Valor M, largura da janela do filtro de médias móveis

    tMin: int
        Primeiro instante desejado [h]

    tMax: int
        Último instante desejado [h]

    Retorna
    -------
    dados_tratados: pd.DataFrame
        Dataset tratado
    """

    dataframe_completo = pd.DataFrame()

    for un in ensaios:
        # Aplica o tratamento apenas se os ensaios podem ser utilizados para a combinação de grandezas desejada
        if (((un['abrev'] == 'b5') and any('Vazao' in string for string in grandezas)) or # B5 tem erro na vazão
            ((un['abrev'] == 'b7') and any('CorrenteRMS' in string for string in grandezas)) or # B7 tem erro na corrente RMS
            ((un['abrev'] == 'b8') and any('VibracaoCalotaInferiorRMS' in string for string in grandezas)) or # B8 não tem dados de vibração
            ((un['abrev'] == 'b8') and any('VibracaoCalotaInferiorCurtose' in string for string in grandezas)) or 
            ((un['abrev'] == 'b8') and any('VibracaoCalotaInferiorVariancia' in string for string in grandezas)) or 
            ((un['abrev'] == 'b8') and any('VibracaoCalotaSuperiorRMS' in string for string in grandezas)) or 
            ((un['abrev'] == 'b8') and any('VibracaoCalotaSuperiorCurtose' in string for string in grandezas)) or 
            ((un['abrev'] == 'b8') and any('VibracaoCalotaSuperiorVariancia' in string for string in grandezas))):
            continue # Pula esta execução e segue para a próxima no for
        
        else: # Ensaios são válidos
            for en in range(len(un['ensaios'])):
                dataframe_ensaio = seleciona_dados(un['ensaios'][en],grandezas,tMin,tMax) # Seleciona os dados
                dataframe_ensaio = rolling_window(dataframe_ensaio, N_instantes, D_espacamento, M_janela) # Aplica rolling window
                dataframe_ensaio["Unidade"] = un['abrev']; dataframe_ensaio["Ensaio"] = 1+en; # Cria uma coluna para o nome da unidade e a ordem do ensaio (Lembre de remover na hora de aplicar ML)

                dataframe_completo = pd.concat([dataframe_completo, dataframe_ensaio]) # Concatena o ensaio atual no dataframe completo

    return dataframe_completo.reset_index() # Retorna o DataFrame com os índices resetados


# Código de exemplo

## Importação de dados

In [28]:
# Importa os ensaios de cada modelo como uma lista de dicionários, importando todos os valores de ensaios;

ensaios_A = [
            {
                'abrev': 'a2',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A2_N_2019_07_09.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A2_A_2019_08_08.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A2_A_2019_08_28.csv')]
            },
            {
                'abrev': 'a3',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A3_N_2019_12_04.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A3_A_2019_12_09.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A3_A_2019_12_11.csv')]
            },
            {
                'abrev': 'a4',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A4_N_2019_12_16.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A4_A_2019_12_19.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A4_A_2020_01_06.csv')]
            },
            {
                'abrev': 'a5',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A5_N_2020_01_22.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A5_A_2020_01_27.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/A5_A_2020_01_28.csv')]
            }
]

ensaios_B = [
            {
                'abrev': 'b5',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B5_N_2020_10_16.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B5_A_2020_10_22.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B5_A_2020_10_27.csv')]
            },
            {
                'abrev': 'b7',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B7_N_2021_02_05.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B7_A_2021_02_08.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B7_A_2021_02_15.csv')]
            },
            {
                'abrev': 'b8',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B8_N_2021_02_18.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B8_A_2021_02_22.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B8_A_2021_02_26.csv')]
            },
            {
                'abrev': 'b10',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B10_N_2021_03_22.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B10_A_2021_03_25.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B10_A_2021_03_30.csv')]
            },
            {
                'abrev': 'b11',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B11_N_2021_04_05.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B11_A_2021_04_08.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B11_A_2021_04_22.csv')]
            },
            {
                'abrev': 'b12',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B12_N_2021_04_27.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B12_A_2021_04_30.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B12_A_2021_05_04.csv')]
            },
            {
                'abrev': 'b15',
                'ensaios': [pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B15_N_2021_05_31.csv'),
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B15_A_2021_06_09.csv'), 
                            pd.read_csv('D:/Documentos/Amaciamento/Apresentações/00_Dissertacao/DadosJoao/B15_A_2021_06_15.csv')]
            },
]

## Pré-processamento de dataset

In [40]:
# Configurações:

tMin = 1 # Instante mínimo do ensaio
tMaxA = 20; tMaxB = 40 # Instante máximo do ensaio

N = 3; D = 1; M = 1; # Confiugrações p/ rolling window

# Grandezas utilizadas no tratamento (Nomes disponíveis:
#                   'CorrenteRMS'               'CorrenteVariancia'            'CorrenteCurtose'
#            'VibracaoCalotaInferiorRMS' 'VibracaoCalotaInferiorCurtose' 'VibracaoCalotaInferiorVariancia'
#            'VibracaoCalotaSuperiorRMS' 'VibracaoCalotaSuperiorCurtose' 'VibracaoCalotaSuperiorVariancia'
#                      'Vazao')
grandezas = ['CorrenteRMS']

In [41]:
# Formação de conjuntos

datasetA = trata_conjunto(ensaios_A, grandezas, N, M, D, tMin, tMaxA) # Processa unidades para um conjunto do modelo A
datasetB = trata_conjunto(ensaios_B, grandezas, N, M, D, tMin, tMaxB) # Processa unidades para um conjunto do modelo B